In [3]:
import sympy as sm
import numpy as np
import os 

print(os.getcwd())

c:\Users\Gilles\CS\cours\PoleProjet\FormationRecherche\Tsunami\TP\sceance4\Tsunami\differentiate


In [2]:
import sympy as sm

x = sm.Symbol('x')

def diff_all_at_once(expr_0):
    return sm.diff(expr_0,x,4)

def diff_gradually(expr_0):
    expr_1 = sm.diff(expr_0,x,1)
    expr_2 = sm.diff(expr_1,x,1)
    expr_3 = sm.diff(expr_2,x,1)
    expr_4 = sm.diff(expr_3,x,1)
    return expr_4

# expr_0 = (x**2-x+1)**20
# print(sm.expand(diff_gradually(expr_0)),'\n \n',sm.expand(diff_all_at_once(expr_0)))
# print(diff_gradually(expr_0)==diff_all_at_once(expr_0)) # this comparison does not work

from timeit import timeit
expr_0 = (x**2-x+1)**20
print(expr_0)
print(timeit(lambda : diff_all_at_once(expr_0),number=1000))
print(timeit(lambda : diff_gradually(expr_0),number=1000))

expr_0 = x**2**20
print(expr_0)
print(timeit(lambda : diff_all_at_once(expr_0),number=1000))
print(timeit(lambda : diff_gradually(expr_0),number=1000))

expr_0 = sm.exp(x**2-1)**20
print(expr_0)
print(timeit(lambda : diff_all_at_once(expr_0),number=1000))
print(timeit(lambda : diff_gradually(expr_0),number=1000))

expr_0 = sm.sin(x**2-1)**20
print(expr_0)
print(timeit(lambda : diff_all_at_once(expr_0),number=1000))
print(timeit(lambda : diff_gradually(expr_0),number=1000))

(x**2 - x + 1)**20
9.394887400000002
0.9346837999999984
x**1048576
0.4959321999999986
0.5476672000000029
exp(20*x**2 - 20)
4.188609400000001
0.6795217999999998
sin(x**2 - 1)**20
13.136855099999998
1.1482468000000026


One can `lambdify` lists but not arrays. 

In [17]:
x,y = sm.symbols('x,y')
variables = [x,y]

A = [[x],[x**2]]
print(sm.diff(A[0][0],x,1))
A = sm.lambdify([x],[[x],[x**2]], 'numpy')
print(A(2))

1
[[2], [4]]


In [31]:
A = [[y,y**2,x],[x**2,[y+1]]]
B = sm.lambdify(variables,A,'numpy')

print(B(1,2))
A[0][0] = x
A[1][1] = y
print(B(1,2)) # no 'effet de bord'
# print(A.evalf(subs={x: 1, y: 2})) => has to choose one A[.][.] before

[[2, 4, 1], [1, [3]]]
[[2, 4, 1], [1, [3]]]


In [5]:
def dynamic_diff_2D(expr_00, l_orders, tab_diff = []):
    if not(len(tab_diff)):
        tab_diff = [[expr_00]]
    for t_order in l_orders:
        a,b=(t_order)
        for i in range(1,a+1):
            if len(tab_diff)<=i:
                tab_diff.append([sm.diff(tab_diff[i-1][0],x)])
        for j in range(1,b+1):
            if len(tab_diff[a])<=j:
                tab_diff[a].append(sm.diff(tab_diff[a][j-1],y))
    return tab_diff[a][b]

def diff_all_at_once_2D(expr_00,t_order):
    dfn = expr_00
    for i, var in enumerate(variables):
        dfn = sm.diff(dfn, var, t_order[i])
    return dfn

In [6]:
from timeit import timeit

t_order = (2,5)

expr_00 = (x**2-x+1+y**3)**20
print(expr_00)
print(timeit(lambda : diff_all_at_once_2D(expr_00,t_order),number=1000))
print(timeit(lambda : dynamic_diff_2D(expr_00,[t_order]),number=1000))

expr_00 = (x*y)**2**20
print(expr_00)
print(timeit(lambda : diff_all_at_once_2D(expr_00,t_order),number=1000))
print(timeit(lambda : dynamic_diff_2D(expr_00,[t_order]),number=1000))

expr_00 = sm.exp(x**2-1+y**3)**20
print(expr_00)
print(timeit(lambda : diff_all_at_once_2D(expr_00,t_order),number=1000))
print(timeit(lambda : dynamic_diff_2D(expr_00,[t_order]),number=1000))

expr_00 = sm.sin(x**2-1+y**3)**20
print(expr_00)
print(timeit(lambda : diff_all_at_once_2D(expr_00,t_order),number=1000))
print(timeit(lambda : dynamic_diff_2D(expr_00,[t_order]),number=1000))


(x**2 - x + y**3 + 1)**20
30.775216300000004
2.325136400000005
x**1048576*y**1048576
0.6674624000000051
0.6651176000000021
exp(20*x**2 + 20*y**3 - 20)
8.795723899999999
2.1101788
sin(x**2 + y**3 - 1)**20
146.22511329999998
5.539002600000003


In [14]:
timeit(lambda: dynamic_diff_2D((x**2-x+1+y**3)**20,[(2,5)]*50),number=1000) # almost the same time (considering deviation)

2.592638299999976